In [1]:
import pandas as pd
import json
import numpy as np
import sys
from dtw import dtw

In [2]:
#algoritmo principale
def DTWDistance(signalA, signalB, channel):
    n,m = len(signalA), len(signalB)
    DTW = np.zeros(shape=(n, m))
    for i in range(0,n):
        DTW[i,0] = np.finfo(dtype=float).max
    for i in range(0,m):
        DTW[0,i] = np.finfo(dtype=float).max
    DTW[0,0] = 0
    
    for i in range(1,n):
        for j in range(1,m):
            const = abs(signalA[i][channel] - signalB[j][channel])
            DTW[i,j] = const + min(DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1])
    
    return DTW[n - 1, m - 1]

In [3]:
#Algoritmo di scaling
def new_scale_df(data_frame, gyro, accl, pres):
    
    def scale(value, max_, min_):
        return (value - min_) / (max_ - min_)
        
    copy = data_frame.copy() 
    
    for wIndex in range(len(copy)):
        for vIndex in range(len(copy[wIndex])):
            copy[wIndex][vIndex][0] = scale(copy[wIndex][vIndex][0], accl, -accl)
            copy[wIndex][vIndex][1] = scale(copy[wIndex][vIndex][1], accl, -accl)
            copy[wIndex][vIndex][2] = scale(copy[wIndex][vIndex][2], accl, -accl)
            copy[wIndex][vIndex][3] = scale(copy[wIndex][vIndex][3], pres, 0)
            copy[wIndex][vIndex][4] = scale(copy[wIndex][vIndex][4], gyro, -gyro)
            copy[wIndex][vIndex][5] = scale(copy[wIndex][vIndex][5], gyro, -gyro)
            copy[wIndex][vIndex][6] = scale(copy[wIndex][vIndex][6], gyro, -gyro)
    
    return copy

In [4]:
def new_shift_df(data_frame, gyro, accl, pres):
    
    def shift(value, shifter):
        return value + shifter
    
    copy = data_frame.copy()
    
    for wIndex in range(len(copy)):
        for vIndex in range(len(copy[wIndex])):
            copy[wIndex][vIndex][0] = shift(copy[wIndex][vIndex][0], accl)
            copy[wIndex][vIndex][1] = shift(copy[wIndex][vIndex][1], accl)
            copy[wIndex][vIndex][2] = shift(copy[wIndex][vIndex][2], accl)
            copy[wIndex][vIndex][4] = shift(copy[wIndex][vIndex][4], gyro)
            copy[wIndex][vIndex][5] = shift(copy[wIndex][vIndex][5], gyro)
            copy[wIndex][vIndex][6] = shift(copy[wIndex][vIndex][6], gyro)
    
    return copy

In [38]:
with open('../dati/giovanni.json', 'r') as f:
    data_giovanni = json.load(f)
with open('../dati/dario.json', 'r') as f:
    data_dario = json.load(f)

In [39]:
scaled_giovanni = new_scale_df(data_giovanni, 250, 2, 1024)
scaled_dario = new_scale_df(data_dario, 250, 2, 1024)

In [7]:
value = [0,0,0,0]
for i in range(7):
    print('indice: ', i)
    l2_norm = lambda x, y: (x[i] - y[i])
    d1, _ , _ , _ = dtw(scaled_giovanni[1], scaled_dario[1], dist=l2_norm)
    print('giovanni vs dario',d1)
    d2, _ , _ , _ = dtw(scaled_giovanni[2], scaled_dario[2], dist=l2_norm)
    print('giovanni vs dario',d2)
    d3, _ , _ , _ = dtw(scaled_giovanni[1], scaled_giovanni[2], dist=l2_norm)
    print('giovanni vs giovanni',d3)
    d4, _ , _ , _ = dtw(scaled_dario[1], scaled_dario[2], dist=l2_norm)
    print('dario vs dario',d4)
    value[0] += d1
    value[1] += d2
    value[2] += d3
    value[3] += d4
value
    

#l2_norm = lambda x, y: (x[0] - y[0]) ** 2
#d1, cost_matrix, acc_cost_matrix, path = dtw(scaled_giovanni[1], scaled_dario[1], dist=l2_norm)
#d2, _ , _ , _ = dtw(scaled_giovanni[1], scaled_giovanni[2], dist=l2_norm)
#print(d1, d2)

indice:  0
antonio vs rocco -0.07972463536776211
antonio vs rocco -0.15479419230769237
antonio vs antonio -0.07488581050000002
rocco vs rocco -0.05472500743162898
indice:  1
antonio vs rocco -0.022810962832550897
antonio vs rocco -0.05149491096866087
antonio vs antonio -0.049843343000000005
rocco vs rocco -0.03024522800237808
indice:  2
antonio vs rocco -0.059611260954616614
antonio vs rocco -0.1640203988603987
antonio vs antonio -0.12415660299999996
rocco vs rocco -0.03729576040428063
indice:  3
antonio vs rocco -0.23856245109546165
antonio vs rocco -0.15968187989672364
antonio vs antonio -0.21171484375
rocco vs rocco -0.23617484765160524
indice:  4
antonio vs rocco 0.0024878560000000226
antonio vs rocco 0.0025071212905983208
antonio vs antonio -0.0009214063919999917
rocco vs rocco -0.0012019134173602902
indice:  5
antonio vs rocco -0.031461395840375626
antonio vs rocco -0.05600789312535612
antonio vs antonio -0.039230411331999976
rocco vs rocco -0.04379352084423306
indice:  6
antonio

[-0.4821524282535211, -0.655904889438034, -0.58110041813, -0.4313913471022591]

In [ ]:
# escludiamo la pressione per ora perché è costante nel tempo e tende a diminuire le distanze fra due utenti

In [40]:
matrix = np.zeros(shape=(len(scaled_dario),len(scaled_dario) + len(scaled_giovanni)))

for i in range(len(scaled_dario)):
    for j in range(len(scaled_dario)):
        for s in range(7):
            if s == 3: continue
            l2_norm = lambda x, y: abs(x[s] - y[s]) 
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_dario[j], dist=l2_norm)
            matrix[i, j] += d
    for j in range(len(scaled_giovanni)):
        for s in range(7):
            if s == 3: continue
            l2_norm = lambda x, y: abs(x[s] - y[s])
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_giovanni[j], dist=l2_norm)
            matrix[i, j + len(scaled_dario)] += d
    print(i)
            
matrix    

0
1
2
3
4
5
6
7
8
9


array([[0.        , 0.09184868, 0.0838361 , 0.10718115, 0.11398469,
        0.11228028, 0.09648448, 0.10166579, 0.09933994, 0.12985454,
        0.09082505, 0.08477525, 0.09365   , 0.09834229, 0.12565454,
        0.12089314, 0.09382096, 0.1122057 ],
       [0.09184868, 0.        , 0.07893817, 0.10691113, 0.10938715,
        0.10407969, 0.09581067, 0.09717719, 0.09167363, 0.12279652,
        0.08022718, 0.07531389, 0.0856993 , 0.09734114, 0.12208693,
        0.11843875, 0.08315687, 0.10969352],
       [0.0838361 , 0.07893817, 0.        , 0.10620095, 0.10942959,
        0.10479639, 0.08884438, 0.09686252, 0.09133093, 0.10432444,
        0.06939479, 0.06761445, 0.07684451, 0.08455727, 0.11809398,
        0.11750907, 0.07248152, 0.10170379],
       [0.10718115, 0.10691113, 0.10620095, 0.        , 0.11574588,
        0.10860839, 0.10624507, 0.109652  , 0.1057191 , 0.16682179,
        0.11407611, 0.10262618, 0.10425693, 0.10850809, 0.13212457,
        0.12389932, 0.11618908, 0.12686189],
    

In [11]:
shifted_dario = new_scale_df(data_dario, 250, 2, 1024)
shifted_giovanni = new_scale_df(data_giovanni, 250, 2, 1024)

In [ ]:
matrix = np.zeros(shape=(len(shifted_dario),len(shifted_dario) + len(shifted_giovanni)))

for i in range(len(shifted_dario)):
    for j in range(len(shifted_dario)):
        l2_norm = lambda x, y: abs((sum(x) - x[3]) - (sum(y) - y[3])) 
        d, _ , _ , _ = dtw(shifted_dario[i], shifted_dario[j], dist=l2_norm)
        matrix[i, j] = d
    for j in range(len(shifted_giovanni)):
        l2_norm = lambda x, y: abs((sum(x) - x[3]) - (sum(y) - y[3]))
        d, _ , _ , _ = dtw(shifted_dario[i], shifted_giovanni[j], dist=l2_norm)
        matrix[i, j + len(shifted_dario)] = d
    print(i)
            
matrix

In [ ]:
matrix = np.zeros(shape=(len(scaled_dario),len(scaled_dario) + len(scaled_giovanni)))
matrix.fill(sys.maxsize)

for i in range(len(scaled_dario)):
    for j in range(len(scaled_dario)):
        for s in range(7):
            l2_norm = lambda x, y: abs(x[s] - y[s]) 
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_dario[j], dist=l2_norm)
            matrix[i, j] = min(matrix[i, j], d)
    for j in range(len(scaled_giovanni)):
        for s in range(7):
            l2_norm = lambda x, y: abs(x[s] - y[s])
            d, _ , _ , _ = dtw(scaled_dario[i], scaled_giovanni[j], dist=l2_norm)
            matrix[i, j + len(scaled_dario)] = min(matrix[i, j + len(scaled_dario)], d)
    print(i)
            
matrix    

In [41]:
d = pd.DataFrame(matrix)
for i in range(len(matrix)):
    print(i,':',(matrix[i]).argmax(), max(matrix[i]))

0 : 9 0.12985453820140513
1 : 9 0.12279651745812803
2 : 14 0.11809397843661967
3 : 9 0.16682179026315783
4 : 9 0.17309891466824645
5 : 9 0.16287671680365298
6 : 9 0.14131020368298366
7 : 9 0.14814506496073906
8 : 9 0.14149996215165878
9 : 4 0.17309891466824645


In [42]:
d

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.000000,0.091849,0.083836,0.107181,0.113985,0.112280,0.096484,0.101666,0.099340,0.129855,0.090825,0.084775,0.093650,0.098342,0.125655,0.120893,0.093821,0.112206
1,0.091849,0.000000,0.078938,0.106911,0.109387,0.104080,0.095811,0.097177,0.091674,0.122797,0.080227,0.075314,0.085699,0.097341,0.122087,0.118439,0.083157,0.109694
2,0.083836,0.078938,0.000000,0.106201,0.109430,0.104796,0.088844,0.096863,0.091331,0.104324,0.069395,0.067614,0.076845,0.084557,0.118094,0.117509,0.072482,0.101704
3,0.107181,0.106911,0.106201,0.000000,0.115746,0.108608,0.106245,0.109652,0.105719,0.166822,0.114076,0.102626,0.104257,0.108508,0.132125,0.123899,0.116189,0.126862
4,0.113985,0.109387,0.109430,0.115746,0.000000,0.113467,0.113174,0.113275,0.109240,0.173099,0.115951,0.103211,0.107822,0.112031,0.128273,0.122380,0.123745,0.132137
5,0.112280,0.104080,0.104796,0.108608,0.113467,0.000000,0.109429,0.106975,0.104766,0.162877,0.109706,0.098576,0.103517,0.109970,0.127729,0.120247,0.115093,0.126650
6,0.096484,0.095811,0.088844,0.106245,0.113174,0.109429,0.000000,0.099950,0.097876,0.141310,0.095401,0.088897,0.094531,0.098646,0.126665,0.116106,0.101361,0.115671
7,0.101666,0.097177,0.096863,0.109652,0.113275,0.106975,0.099950,0.000000,0.101118,0.148145,0.101006,0.091451,0.097683,0.103045,0.124287,0.117825,0.104780,0.118353
8,0.099340,0.091674,0.091331,0.105719,0.109240,0.104766,0.097876,0.101118,0.000000,0.141500,0.095587,0.089083,0.090845,0.097747,0.124101,0.117046,0.100683,0.110559
9,0.129855,0.122797,0.104324,0.166822,0.173099,0.162877,0.141310,0.148145,0.141500,0.000000,0.083282,0.086586,0.102861,0.112554,0.162449,0.165782,0.078916,0.139501


In [43]:
min_values, max_values = [], []
for i in range(len(matrix)):
    minv, maxv = (sys.maxsize, -1), (-sys.maxsize, -1)
    for j in range(len(matrix[i])):
        if i == j: 
            continue
        minv = (matrix[i][j], j) if minv[0] > matrix[i][j] else minv
        maxv = (matrix[i][j], j) if maxv[0] < matrix[i][j] else maxv
    min_values.append(minv)
    max_values.append(maxv)
    
for i in range(len(min_values)):
    print("index ", i, " min value: ", min_values[i])
        

index  0  min value:  (0.08383609945454545, 2)
index  1  min value:  (0.075313891401084, 11)
index  2  min value:  (0.0676144507827225, 11)
index  3  min value:  (0.1026261841979695, 11)
index  4  min value:  (0.1032111794774535, 11)
index  5  min value:  (0.09857643924675324, 11)
index  6  min value:  (0.08884438374244835, 2)
index  7  min value:  (0.09145092260130716, 11)
index  8  min value:  (0.08908338864986738, 11)
index  9  min value:  (0.07891607263984678, 16)
